# Benchmark against other methods


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pickle
import json
import seaborn as sns
import matplotlib.transforms as mtransforms
from bread import vis
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bread.algo import tracking
from bread.data import *
from sklearn.metrics import confusion_matrix
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
from glob import glob
import sys
import os.path
sys.path.append('../../src/libraries')  # import YeaZ

## Load testing data


In [4]:
seg = SegmentationFile.from_h5(
    '../../data/colony000_corrected/segmentation_colony000_corrected.h5').get_segmentation("FOV0")
microscopy = Microscopy.from_tiff(
    '../../data/data_edited/colony000_microscopy.tif')
feat = Features(seg, nn_threshold=12, scale_length=1, scale_time=1)

In [5]:
max_dt = 1
# only test for first seg_len frames (assume only #seg_len frames are available)
seg_len = 10
seg_start = 116
interframes = [(idt, idt+dt) for idt, dt in itertools.product(
    range(seg_start, seg_start+seg_len), range(1, max_dt+1)) if idt+dt < seg_start+seg_len]
print(f'{len(interframes)} interframe samples')

9 interframe samples


In [6]:
results = {'gcn': {}, 'yeaz': {}, 'tracx': {}}

In [7]:
recompute = True  # notebook flag to force recomputing the accuracies

In [8]:
# directory for results
basedir = Path('../../data/generated/tracking/compare/colony000_debug')

# find average cell area in first frames


In [12]:
area_list = []
for i in range(1, 4):
    seg = SegmentationFile.from_h5(
        '../../data/data_edited/colony00{}_segmentation.h5'.format(i)).get_segmentation("FOV0")
    microscopy = Microscopy.from_tiff(
        '../../data/data_edited/colony00{}_microscopy.tif'.format(i))
    feat = Features(seg, nn_threshold=12, scale_length=1, scale_time=1)
    cells = seg.cell_ids(0)
    for cell in cells:
        area_list.append(feat.cell_area(0, cell))

In [13]:

print(np.mean(area_list))
area_list

2078.181818181818


[3597.5,
 2005.5,
 1897.0,
 731.5,
 2669.5,
 2836.0,
 728.0,
 3217.5,
 1774.5,
 486.5,
 2916.5]

## Our method (GCN)


In [8]:
modelpath = Path('../../data/generated/tracking/models/2023-06-14 14:30:22/')

### Load model from pickle


In [9]:
with open(modelpath / 'pickle.pkl', 'rb') as file:
    net: tracking.AssignmentClassifier = pickle.load(file)

net.trim_for_prediction()
net

<class 'bread.algo.tracking._classifier.AssignmentClassifier'>[initialized](
  module_=GNNTracker(
    (node_encoder): MLP(12, 120, 120, 120, 120)
    (edge_encoder): MLP(10, 120, 120, 120, 120)
    (layers): ModuleList(
      (0-4): 5 x DeepGCNLayer(block=res+)
    )
    (out): MLP(120, 60, 30, 1)
  ),
)

### Track a custom frame


In [31]:
ga = tracking.build_assgraph(tracking.build_cellgraph(
    feat, 122), tracking.build_cellgraph(feat, 121), include_target_feature=True)
gat, *_ = tracking.to_data(ga)
yhat = net.predict_assignment(gat)
yhat.shape

(185, 176)

In [32]:
def matrix_to_dict(matrix):
    n = matrix.shape[1]
    result_dict = {}
    for col in range(n):
        row_indices = np.where(matrix[:, col] == 1)[0]
        if len(row_indices) == 0:
            result_dict[col] = -1
        else:
            result_dict[col] = row_indices[0]
    return result_dict


ass_dict = matrix_to_dict(yhat)

In [ ]:
# generate segmentation from the y_hat matrix
def correspondence(seg, prev, next_frame, y_mat):
    newcell = np.max(seg.cell_ids(prev)) + 1

    ass_dict = matrix_to_dict(y_mat)
    # curr and new are basically same thing. we just need to have a copy of curr to edit it.
    curr = seg.get_frame(next_frame, fov='FOV0')
    new = seg.get_frame(next_frame, fov='FOV0')
    for key, val in ass_dict.items():
        # If new cell
        if val == -1:
            val = newcell
            newcell += 1

        new[curr == key] = val

    return new

In [ ]:
x = correspondence()

In [33]:
x = pd.DataFrame()
x['prev'] = ass_dict.values()
x['next'] = ass_dict.keys()
x.loc[x.prev != x.next]

,prev,next
104,142,104
114,147,114
117,114,117
130,104,130
142,182,142
147,179,147
160,184,160
162,180,162


In [34]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [35]:
seg.cell_ids(122)[104]
seg.cell_ids(122)[114]
seg.cell_ids(122)[85]
seg.cell_ids(123)[104]
seg.cell_ids(123)[114]
seg.cell_ids(123)[85]

109

119

89

109

119

89

### compute


In [83]:
if recompute:
    results['gcn'] = {
        't1': [],
        't2': [],
        'confusion': []
    }

    for idt1, idt2 in tqdm(interframes):
        ga = tracking.build_assgraph(tracking.build_cellgraph(
            feat, idt1), tracking.build_cellgraph(feat, idt2), include_target_feature=True)
        gat, *_ = tracking.to_data(ga)

        yhat = net.predict_assignment(gat).flatten()
        y = gat.y.squeeze().cpu().numpy()

        results['gcn']['confusion'].append(confusion_matrix(y, yhat))
        results['gcn']['t1'].append(idt1)
        results['gcn']['t2'].append(idt2)

 32%|███▏      | 6/19 [05:10<11:47, 54.45s/it]/Users/farzanehwork/Documents/codes/bread/src/bread/data/_data.py:603: MultipleContoursWarning: OpenCV returned multiple contours, 2 found.
  warnings.warn(Contour.MultipleContoursWarning(len(contours_cv)))
100%|██████████| 19/19 [24:38<00:00, 77.80s/it] 


### Test timmings


In [84]:
idt1, idt2 = 40, 43

In [85]:
ga = tracking.build_assgraph(tracking.build_cellgraph(
    feat, idt1), tracking.build_cellgraph(feat, idt2), include_target_feature=True)

In [86]:
gat, *_ = tracking.to_data(ga)

In [87]:


yhat = net.predict_assignment(gat).flatten()

In [88]:

y = gat.y.squeeze().cpu().numpy()

results['gcn']['confusion'].append(confusion_matrix(y, yhat))
results['gcn']['t1'].append(idt1)
results['gcn']['t2'].append(idt2)

## YeaZ


In [89]:
import YeaZ.unet.hungarian as YeaZ_hung

In [90]:
import scipy.spatial.distance


def match_cellids_from_segs(segframe1: np.ndarray, segframe2: np.ndarray, eps=1e-6):
    # Matches cellids between two identical segmentations, which use different numberings
    seg1 = Segmentation(segframe1[None, ...], fov='FOV0')
    seg2 = Segmentation(segframe2[None, ...], fov='FOV0')
    cms1 = seg1.cms(0)
    cms2 = seg2.cms(0)

    # compare centers of masses
    dists = scipy.spatial.distance.cdist(cms1, cms2)
    matches = dists < eps
    idx1, idx2 = matches.nonzero()
    assert not len(idx2) < len(cms1), 'not all cells were matched'
    assert not len(idx2) > len(cms1), 'too many cells were matched'

    return idx2

In [91]:
if recompute:
    results['yeaz'] = {
        't1': [],
        't2': [],
        'confusion': []
    }

    for idt1, idt2 in tqdm(interframes):
        seg_hat = YeaZ_hung.correspondence(seg[idt1], seg[idt2])
        idmap2 = match_cellids_from_segs(seg[idt2], seg_hat)

        y = tracking.assignment_from_segmentation(seg, idt1, idt2)
        yhat = tracking.assignment_from_segmentation(seg, idt1, idt2)[
            :, idmap2]

        results['yeaz']['confusion'].append(
            confusion_matrix(y.flatten(), yhat.flatten()))
        results['yeaz']['t1'].append(idt1)
        results['yeaz']['t2'].append(idt2)

100%|██████████| 19/19 [02:21<00:00,  7.43s/it]


## TracX


In [92]:
# since tracx works with matlab, we need to export the data to a convenient format
# then run it via matlab
# then analyze the results

import tifffile

fileid2frameid = defaultdict(dict)
dowrite = False

for dt in range(1, max_dt+1):
    outdir = str(basedir / 'TracX/framediff{:02d}').format(dt)
    os.makedirs(outdir, exist_ok=True)
    fileid = 0
    for startframe in range(0, dt):
        for frameid in range(startframe, seg_len, dt):
            fileid2frameid[dt][fileid] = frameid
            if dowrite:
                tifffile.imsave(
                    f'{outdir}/mask_{fileid:03d}.tif', seg[frameid])
                tifffile.imsave(
                    f'{outdir}/phc_{fileid:03d}.tif', microscopy[frameid])
            fileid += 1

if dowrite:
    with open(basedir / 'TracX/fileid2frameid.json', 'w') as f:
        json.dump(fileid2frameid, f)

In [93]:
# run the tracking in matlab and generate results in basedir / TracX_results/ ...

In [94]:
dfs_tracx_res = []

for framediff in range(1, max_dt+1):
    filepath = basedir / \
        f'TracX_results/TrackingResults_framediff{framediff:02d}.txt'
    df = pd.read_csv(filepath, sep='\t', usecols=[
                     'cell_frame', 'cell_index', 'track_index'])
    df['frameid'] = df.cell_frame.map(lambda f: fileid2frameid[1][f-1])
    df.drop(columns='cell_frame', inplace=True)
    df['framediff'] = framediff
    dfs_tracx_res.append(df)

df_tracx_res = pd.concat(dfs_tracx_res)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/generated/tracking/compare/colony000_debug/TracX_results/TrackingResults_framediff01.txt'

In [ ]:
if recompute:
    results['tracx'] = {
        't1': [],
        't2': [],
        'confusion': []
    }

    for _, group_framediff in df_tracx_res.groupby('framediff'):
        framediff = group_framediff.framediff.iloc[0]

        for (_, group1), (_, group2) in itertools.product(group_framediff.groupby('frameid'), repeat=2):
            idt1, idt2 = group1.frameid.iloc[0], group2.frameid.iloc[0]

            # loop only over idt -> idt+framediff
            if idt2 - idt1 != framediff or idt2 < idt1:
                continue

            ass_tracx = np.empty(
                (len(group1.cell_index), len(group2.cell_index)))
            ass = np.empty((len(group1.cell_index), len(group2.cell_index)))

            for idx1, idx2 in itertools.product(range(len(group1.cell_index)), range(len(group2.cell_index))):
                ass_tracx[idx1, idx2] = group1.track_index.iloc[idx1] == group2.track_index.iloc[idx2]
                # the input to TracX are segmentations where the cellids are already tracked correctly
                # so the tracking is just where the cell indices match
                ass[idx1, idx2] = group1.cell_index.iloc[idx1] == group2.cell_index.iloc[idx2]

            results['tracx']['confusion'].append(
                confusion_matrix(ass.flatten(), ass_tracx.flatten()))
            results['tracx']['t1'].append(idt1)
            results['tracx']['t2'].append(idt2)

## Compare results


### Create results dataframe


In [1]:
resultsfile = basedir / 'results.csv'

if recompute:
    dfs = []

    for method in results:
        res = pd.DataFrame(results[method])
        res['method'] = method
        dfs.append(res)

    res = pd.concat(dfs, ignore_index=True)

    num_cells = [len(seg.cell_ids(idt)) for idt in range(seg_len)]

    res['tp'] = res['confusion'].map(lambda c: c[1, 1])
    res['fp'] = res['confusion'].map(lambda c: c[0, 1])
    res['tn'] = res['confusion'].map(lambda c: c[0, 0])
    res['fn'] = res['confusion'].map(lambda c: c[1, 0])
    res['acc'] = (res['tp'] + res['tn']) / \
        (res['tp'] + res['fp'] + res['tn'] + res['fn'])
    res['f1'] = 2*res['tp'] / (2*res['tp'] + res['fp'] + res['fn'])
    res['num_cells1'] = res['t1'].map(lambda t: num_cells[int(t)])
    res['num_cells2'] = res['t2'].map(lambda t: num_cells[int(t)])
    res['timediff'] = 5 * (res['t2'] - res['t1'])
    res.drop(columns='confusion', inplace=True)
    res.to_csv(resultsfile)

else:
    res = pd.read_csv(resultsfile, index_col=0)

NameError: name 'basedir' is not defined

In [ ]:
resultsfile

In [98]:
f_p = 119
# sort by t1
res = res.sort_values(by='t1')
res

,t1,t2,method,tp,fp,tn,fn,acc,f1,num_cells1,num_cells2,timediff
19,40.0,43.0,gcn,14,0,182,0,1.000000,1.000000,14,14,15.0
20,118.0,119.0,yeaz,162,3,27552,3,0.999784,0.981818,165,168,5.0
0,118.0,119.0,gcn,165,0,27555,0,1.000000,1.000000,165,168,5.0
1,119.0,120.0,gcn,165,3,28893,3,0.999794,0.982143,168,173,5.0
21,119.0,120.0,yeaz,165,3,28893,3,0.999794,0.982143,168,173,5.0
22,120.0,121.0,yeaz,173,0,30275,0,1.000000,1.000000,173,176,5.0
2,120.0,121.0,gcn,173,0,30275,0,1.000000,1.000000,173,176,5.0
23,121.0,122.0,yeaz,176,0,32384,0,1.000000,1.000000,176,185,5.0
3,121.0,122.0,gcn,171,5,32379,5,0.999693,0.971591,176,185,5.0
4,122.0,123.0,gcn,182,3,35147,3,0.999830,0.983784,185,191,5.0


### Sanity checks


In [ ]:
res.groupby(['timediff', 'method'])[['acc']].count()

In [ ]:
res.groupby(['method', 'timediff'])[['acc']].sum()

In [ ]:
res.groupby(['method', 'timediff'])[['acc']].mean()

### Evaluate CI & plot


In [ ]:
from scipy.stats import bootstrap

In [ ]:
def frac_perfect(col):
    return (col == 1.0).sum() / len(col)

In [ ]:
def frac_perfect_ci(col):
    # we need to test this otherwise scipy complains about degeneracy
    if all(col == 1):
        return (1.0, 1.0)
    bs = bootstrap((col,), statistic=frac_perfect,
                   vectorized=False, random_state=0)
    return bs.confidence_interval.low, bs.confidence_interval.high


def mean_ci(col):
    if all(col == 1):
        return (1.0, 1.0)
    bs = bootstrap((col,), statistic=np.mean, vectorized=False, random_state=0)
    return bs.confidence_interval.low, bs.confidence_interval.high

In [ ]:
res_agg = (res
           .groupby(['timediff', 'method'])
           [['acc']]
           .agg([frac_perfect, frac_perfect_ci, np.mean, mean_ci, np.median])
           .reset_index()
           )
res_agg[[('acc', 'frac_perfect_ci_lower'), ('acc', 'frac_perfect_ci_higher')]] = pd.DataFrame(
    res_agg[('acc', 'frac_perfect_ci')].tolist(), index=res_agg.index)
res_agg[[('acc', 'mean_ci_lower'), ('acc', 'mean_ci_higher')]] = pd.DataFrame(
    res_agg[('acc', 'mean_ci')].tolist(), index=res_agg.index)
res_agg.drop(columns=[('acc', 'mean_ci'),
             ('acc', 'frac_perfect_ci')], inplace=True)
res_agg

In [ ]:
fig, axs = plt.subplot_mosaic([['a)'], ['b)'], ['c)']], figsize=(
    4, 6), sharex=True, tight_layout=True)

axs['b)'].axhline(1.0, color='tab:gray')
axs['b)'].fill_between(res_agg.timediff[res_agg.method == 'gcn'], res_agg.acc.frac_perfect_ci_lower[res_agg.method == 'gcn'], res_agg.acc.frac_perfect_ci_higher[res_agg.method == 'gcn'],
                       alpha=0.2)
axs['b)'].plot(res_agg.timediff[res_agg.method == 'gcn'], res_agg.acc.frac_perfect[res_agg.method == 'gcn'],
               'o--', markersize=4, linewidth=1,
               label='GCN', clip_on=False)
axs['b)'].fill_between(res_agg.timediff[res_agg.method == 'yeaz'], res_agg.acc.frac_perfect_ci_lower[res_agg.method == 'yeaz'], res_agg.acc.frac_perfect_ci_higher[res_agg.method == 'yeaz'],
                       alpha=0.2)
axs['b)'].plot(res_agg.timediff[res_agg.method == 'yeaz'], res_agg.acc.frac_perfect[res_agg.method == 'yeaz'],
               'o--', markersize=4, linewidth=1,
               label='Hungarian (YeaZ)', clip_on=False)
axs['b)'].fill_between(res_agg.timediff[res_agg.method == 'tracx'], res_agg.acc.frac_perfect_ci_lower[res_agg.method == 'tracx'], res_agg.acc.frac_perfect_ci_higher[res_agg.method == 'tracx'],
                       alpha=0.2)
axs['b)'].plot(res_agg.timediff[res_agg.method == 'tracx'], res_agg.acc.frac_perfect[res_agg.method == 'tracx'],
               'o--', markersize=4, linewidth=1,
               label='TracX', clip_on=False)
# axs['b)'].set_xlabel('Time difference between frames [minutes]')
axs['b)'].set_ylabel('Fraction of perfect\ntrackings (acc=1)')
axs['b)'].spines.top.set_visible(False)
axs['b)'].spines.right.set_visible(False)
# axs['b)'].legend(fontsize='small', frameon=False)
axs['b)'].set_ylim((None, 1.0))

axs['c)'].axhline(1.0, color='tab:gray')
axs['c)'].fill_between(res_agg.timediff[res_agg.method == 'gcn'], res_agg.acc['mean_ci_lower'][res_agg.method == 'gcn'], res_agg.acc['mean_ci_higher'][res_agg.method == 'gcn'],
                       alpha=0.2)
axs['c)'].plot(res_agg.timediff[res_agg.method == 'gcn'], res_agg.acc['mean'][res_agg.method == 'gcn'],
               'o--', markersize=4, linewidth=1,
               label='GCN', clip_on=False)
axs['c)'].fill_between(res_agg.timediff[res_agg.method == 'yeaz'], res_agg.acc['mean_ci_lower'][res_agg.method == 'yeaz'], res_agg.acc['mean_ci_higher'][res_agg.method == 'yeaz'],
                       alpha=0.2)
axs['c)'].plot(res_agg.timediff[res_agg.method == 'yeaz'], res_agg.acc['mean'][res_agg.method == 'yeaz'],
               'o--', markersize=4, linewidth=1,
               label='Hungarian (YeaZ)', clip_on=False)
axs['c)'].fill_between(res_agg.timediff[res_agg.method == 'tracx'], res_agg.acc['mean_ci_lower'][res_agg.method == 'tracx'], res_agg.acc['mean_ci_higher'][res_agg.method == 'tracx'],
                       alpha=0.2)
axs['c)'].plot(res_agg.timediff[res_agg.method == 'tracx'], res_agg.acc['mean'][res_agg.method == 'tracx'],
               'o--', markersize=4, linewidth=1,
               label='TracX', clip_on=False)
axs['c)'].set_ylabel('Mean accuracy')
axs['c)'].spines.top.set_visible(False)
axs['c)'].spines.right.set_visible(False)
axs['c)'].legend(fontsize='small', frameon=False)
axs['c)'].set_ylim((None, 1.0))
axs['c)'].set_xlabel('Time difference between frames [minutes]')

sns.stripplot(
    data=res, x='timediff', y='acc', hue='method',
    dodge=True, s=3, alpha=0.7, ax=axs['a)'], legend=False, native_scale=True,
    # clip_on=False,
)
axs['a)'].axhline(1.0, color='tab:gray')
# some outliers from YeaZ and trackx are not shown !!
axs['a)'].set_ylim((0.97, 1))
axs['a)'].set_ylabel('Accuracy')
axs['a)'].set_xlabel('Time difference between frames [minutes]')
axs['a)'].set_xticks(res_agg.timediff.unique())
axs['a)'].spines.top.set_visible(False)
axs['a)'].spines.right.set_visible(False)

for label, ax in axs.items():
    # label physical distance in and down
    trans = mtransforms.ScaledTranslation(-0.7, 0.05, fig.dpi_scale_trans)
    ax.text(0.0, 1.0, label, transform=ax.transAxes + trans,
            fontsize='medium', verticalalignment='bottom',
            bbox=dict(facecolor='0.7', edgecolor='none', pad=3.0))

# fig.savefig('../report_gnn/figures/timediff_accuracy.pdf')
plt.show()

### Evaluate p-value from bootstrapped samples


In [ ]:
from typing import Callable
from scipy.stats import ttest_ind


def bootstrap_sample(
        x: np.ndarray,
        n: int = 10_000,
        agg: Callable[[np.ndarray], float] = np.mean
) -> np.ndarray:
    """Compute confidence interval of a given data aggregate

    Parameters
    ----------
    x : np.ndarray
            data
    alpha : float, optional
            size of the confidence interval, by default 0.95
    n : int, optional
            number of resamples, by default 1_000
    agg : Callable[[np.ndarray], float], optional
            aggregation function, by default np.mean

    Returns
    -------
    aggs: np.ndarray of length ``n``
    """

    aggs = np.zeros(n)

    for i in range(n):
        # resample data uniformly
        sample = x[np.random.randint(0, len(x), len(x))]
        aggs[i] = agg(sample)

    return np.sort(aggs)

In [ ]:
fig, axs = plt.subplots(nrows=res.timediff.nunique(),
                        figsize=(5, 20), constrained_layout=True)

for ax, timediff in zip(axs, res.timediff.unique()):
    print(f'-- timediff = {timediff} --')

    # we resample ~180 times because that's the number of datapoints we have
    sample_gcn = bootstrap_sample(res.query(
        'timediff == @timediff and method == "gcn"').acc.to_numpy(), n=180, agg=frac_perfect)
    sample_tracx = bootstrap_sample(res.query(
        'timediff == @timediff and method == "tracx"').acc.to_numpy(), n=180, agg=frac_perfect)
    sample_yeaz = bootstrap_sample(res.query(
        'timediff == @timediff and method == "yeaz"').acc.to_numpy(), n=180, agg=frac_perfect)

    sns.histplot(dict(gcn=sample_gcn, tracx=sample_tracx,
                 yeaz=sample_yeaz), binwidth=0.005, ax=ax)
    ax.set_title(f'timediff={timediff} min')

    print(f't-test gcn <-> tracx : {ttest_ind(sample_gcn, sample_tracx)}')
    print(f't-test gcn <-> yeaz : {ttest_ind(sample_gcn, sample_yeaz)}')

## What induces mistakes ?


In [ ]:
for _, row in res.query('(method == "gcn") and (fp > 0 or fn > 0)').sort_values('acc').iterrows():
    fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
    vis.plot_visible(microscopy, row.t1, figax=(fig, axs[0]))
    vis.plot_segmentation(seg, row.t1, figax=(
        fig, axs[0]), cellids=True, alpha=0.2)
    vis.plot_visible(microscopy, row.t2, figax=(fig, axs[1]))
    vis.plot_segmentation(seg, row.t2, figax=(
        fig, axs[1]), cellids=True, alpha=0.2)
    fig.suptitle(f'{row.t1} -> {row.t2} (dt={row.timediff} mins)')
    ga = tracking.build_assgraph(tracking.build_cellgraph(
        feat, row.t1), tracking.build_cellgraph(feat, row.t2), include_target_feature=True)
    gat, *_ = tracking.to_data(ga)
    yhat = net.predict_assignment(gat)
    y = gat.y.cpu().numpy().reshape(yhat.shape)
    cellids1, cellids2 = seg.cell_ids(row.t1), seg.cell_ids(row.t2)
    coloriter = iter(plt.rcParams['axes.prop_cycle'].by_key()['color'])
    for idx1, (idx2, idx2_hat) in enumerate(zip(np.argmax(y, axis=1), np.argmax(yhat, axis=1))):
        if idx2 != idx2_hat:
            color = next(coloriter)
            print(
                f'Expected {cellids1[idx1]}->{cellids2[idx2]}, got {cellids1[idx1]}->{cellids2[idx2_hat]}')
            vis._vis._plot_ellipse(feat._ellipse(cellids1[idx1], row.t1), figax=(
                fig, axs[0]), color=color, linewidth=2, alpha=0.8, draw_axes=False)
            vis._vis._plot_ellipse(feat._ellipse(cellids2[idx2], row.t2), figax=(
                fig, axs[1]), color=color, linewidth=2, alpha=0.8, draw_axes=False)
            vis._vis._plot_ellipse(feat._ellipse(cellids2[idx2_hat], row.t2), figax=(
                fig, axs[1]), color=color, linestyle='--', linewidth=2, alpha=0.8, draw_axes=False)
    plt.show()